# 数据规整：聚合、合并和重塑

本章关注聚合、合并和重塑数据的方法。

## 层次化索引

层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理高维度数据。

In [23]:
import numpy as np
import pandas as pd

In [24]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], # 传入双层列表作为索引
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.554467
   2    0.136919
   3    0.532146
b  1   -1.683631
   3    1.269165
c  1   -1.582712
   2    0.366023
d  2    2.332823
   3    0.029179
dtype: float64

In [25]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [26]:
data['b'] # 选取也是多层

1   -1.683631
3    1.269165
dtype: float64

In [27]:
data['b':'c']

b  1   -1.683631
   3    1.269165
c  1   -1.582712
   2    0.366023
dtype: float64

In [28]:
data.loc[['b', 'd']]

b  1   -1.683631
   3    1.269165
d  2    2.332823
   3    0.029179
dtype: float64

In [29]:
# 内层中选取
data.loc[:,2]

a    0.136919
c    0.366023
d    2.332823
dtype: float64

In [30]:
data.unstack()
# 层次索引在数据重塑和基于分组的操作中扮演着重要角色

,1,2,3
a,-0.554467,0.136919,0.532146
b,-1.683631,NaN,1.269165
c,-1.582712,0.366023,NaN
d,NaN,2.332823,0.029179


In [31]:
data.unstack().stack()

a  1   -0.554467
   2    0.136919
   3    0.532146
b  1   -1.683631
   3    1.269165
c  1   -1.582712
   2    0.366023
d  2    2.332823
   3    0.029179
dtype: float64

In [66]:
frame = pd.DataFrame(np.arange(12).reshape(4, 3),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [33]:
# 各层也可以指定名字
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [34]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [67]:
MultiIndex.from_arrays(
    [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
    names=['state', 'color'])

NameError: name 'MultiIndex' is not defined

### 重排和分级索引

In [36]:
frame.swaplevel('key1', 'key2') #交换 key1 和 key2

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [68]:
frame.sort_index(level=1) # level 1： key2

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
b 1     6   7        8
a 2     3   4        5
b 2     9  10       11

In [38]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 根据级别汇总统计

In [39]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [40]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 使用 DataFrame 的列进行索引

In [42]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame                 

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [43]:
frame2 = frame.set_index(['c', 'd'])
frame2 # 默认被选的列会删除

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [44]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [46]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 合并数据集

- pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来。SQL或其他关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的join操作。
- pandas.concat可以沿着一条轴将多个对象堆叠到一起。
- 实例方法combine_first可以将重复数据拼接在一起，用一个对象中的值填充另一个对象中的缺失值。


### 数据库风格的DataFrame合并

数据集的合并（merge）或连接（join）运算是通过一个或多个键将行连接起来的。这些运算是关系型数据库（基于SQL）的核心。pandas的merge函数是对数据应用这些算法的主要切入点。

不同的连接类型：
![表8-1 不同的连接类型](Img/7178691-e49b3341f4a3c90e.png)

merge函数的参数：
![](Img/7178691-35ca716a4f1b8475.png)
![](Img/7178691-c86672e733ceccd9.png)
indicator 添加特殊的列_merge，它可以指明每个行的来源，它的值有left_only、right_only或both，根据每行的合并数据的来源。

In [47]:
df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [48]:
df1 # key 多个abcd

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [49]:
df2 # 只有a、b、d

,key,data2
0,a,0
1,b,1
2,d,2


In [50]:
pd.merge(df1,df2) # m:1
# 如果不指定列，merge会将重叠列的列名当做键

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [51]:
pd.merge(df1,df2,on='key') # 指定列

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [53]:
# 要合并的列不同，分别指定
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                   'data2': range(3)})  

In [54]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

# c,d 为何消失了？
# 默认情况下，merge做的是“内连接”，结果中的键是交集。
# 其他方式还有 left、right和outer
# 外连接求取的是键的并集，组合了左连接和右连接的效果

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [55]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [56]:
# 多对多合并
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})  

In [57]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [58]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [59]:
pd.merge(df1,df2,on='key',how='left')

# 多对多连接产生的是行的笛卡尔积。
# 由于左边的DataFrame有3个"b"行，右边的有2个，所以最终结果中就有6个"b"行。

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [60]:
pd.merge(df1, df2, how='inner')
# 连接方式只影响出现在结果中的不同的键的值

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [62]:
# 多个键合并，传入一个由列名组成的列表即可
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [63]:
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [64]:
# 合并中，重复列名的处理
pd.merge(left, right, on='key1')
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7
